# Full Bonds Calculator

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import datetime
import math as m
from dateutil import relativedelta
%matplotlib inline

In [5]:
def create_dates(start, end, frequency):
    d1=datetime.datetime.strptime(start,"%m/%d/%Y")
    d2=datetime.datetime.strptime(end,"%m/%d/%Y")
    
    years=int(relativedelta.relativedelta(d2,d1).years)
    dates_list=[]
    
    for t in range(0,frequency*years):
        x=d1 + relativedelta.relativedelta(months=(12/frequency)*t)
        dates_list.append(x.strftime('%m/%d/%Y'))
    
    if end not in dates_list:
        dates_list.append(end)
    
    return dates_list

class Bond ():
    def __init__(self, RV, dates,coupon,amort_pay, price, K):
        self.coupon=coupon
        self.dates=dates
        self.dates.insert(0, (datetime.date.today()).strftime('%m/%d/%Y'))
        self.RV=RV
        self.price=price
        self.amort_pay=amort_pay
        self.K=K
    
    def interest_payments(self):
        years_coupon=[]
        for t in range(1,len(self.dates)):
            d=self.dates[t]
            h=self.dates[t-1]
            years_coupon.append((datetime.datetime.strptime(d,"%m/%d/%Y")-
                          datetime.datetime.strptime(h,"%m/%d/%Y")).days/365)
        ilist=[]
        for t in range(0,len(years_coupon)):
            y=years_coupon[t]
            if y>0:
                ilist.append((self.coupon*years_coupon[t])*self.RV)
            else:
                ilist.append(0)
        ilist.insert(0,0)
        return ilist

    def amort_payments(self):
        amort_flow=[]
        if self.amort_pay[0]==None:
            for t in range(0,len(self.dates)):
                x=self.dates[t]
                if x==self.dates[-1]:
                    amort_flow.append(self.RV)
                else:
                    amort_flow.append(0)
            return amort_flow
        elif self.amort_pay[0]!=None:
            self.amort_pay.insert(0,0)
            return self.amort_pay
    
    def cflows(self):
        principal_cf=self.amort_payments()
        interest_cf=self.interest_payments()
        cash_flows=[]
        for t in range(0,len(self.dates)):
            cash_flows.append(principal_cf[t]+interest_cf[t])
        cash_flows[0]=-self.price
        return cash_flows
    
    def date_to_years(self):
        dlist=[]
        for t in range(0,len(self.dates)):
            d=self.dates[t]
            dlist.append(round((datetime.datetime.strptime(d,"%m/%d/%Y")-
                          datetime.datetime.now()).days/365,4))
        dlist[0]=0
        return dlist
    
    def IRR(self):
        CFS=self.cflows()
        years=self.date_to_years()
        x0=1
        return np.asscalar(fsolve(NPV, x0=x0, args=(np.array(CFS),
                                                        np.array(years))))
    def PV(self):
        CFS=self.cflows()
        years=self.date_to_years()
        IRR=self.K
        return np.sum(np.array(CFS[1:])/(1+IRR)**np.array(years[1:]))

def NPV(IRR,CFS,years):
    return np.sum(np.array(CFS)/(1+IRR)**np.array(years))

In [6]:
xbond=Bond(100, ['2/4/2021','5/4/2021','8/4/2021','11/4/2021','2/4/2022'],0.1,[None],80,0.3)
pd.DataFrame({'Date':xbond.dates,'In years':xbond.date_to_years(),
              'Interest payments':xbond.interest_payments(), 
              'Principal payments':xbond.amort_payments(), 'Cash flow':xbond.cflows()})

,Date,In years,Interest payments,Principal payments,Cash flow
0,01/08/2021,0.0000,0.000000,0,-80.000000
1,2/4/2021,0.0712,0.739726,0,0.739726
2,5/4/2021,0.3151,2.438356,0,2.438356
3,8/4/2021,0.5671,2.520548,0,2.520548
4,11/4/2021,0.8192,2.520548,0,2.520548
5,2/4/2022,1.0712,2.520548,100,102.520548
